In [106]:
import pandas as pd
import numpy as np
import os
from tensorflow import keras
import cv2
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping

In [107]:
train_set = pd.read_csv("./train_set.csv")
train_set

,id_img,paths,label
0,0,data\train\angry\0.jpg,angry
1,1,data\train\angry\1.jpg,angry
2,10,data\train\angry\10.jpg,angry
3,10002,data\train\angry\10002.jpg,angry
4,10016,data\train\angry\10016.jpg,angry
...,...,...,...
28816,9969,data\train\surprise\9969.jpg,surprise
28817,9985,data\train\surprise\9985.jpg,surprise
28818,9990,data\train\surprise\9990.jpg,surprise
28819,9992,data\train\surprise\9992.jpg,surprise


In [108]:
test_set = pd.read_csv("./test_set.csv")
test_set

,id_img
0,10052
1,10065
2,10079
3,10095
4,10121
...,...
7061,9806
7062,9830
7063,9853
7064,9878


In [109]:
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
class_names_label

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [110]:
TRAIN_PATH = os.getcwd() + '\\data\\train'
TEST_PATH = os.getcwd() + '\\data\\test'
IMAGE_SIZE = (32,32)

In [111]:
def read_data(path, im_size):
    X = []
    Y = []

    for folder in os.listdir(path):
        label = class_names_label[folder]
        folder_path = os.path.join(path, folder)

    ##### CODE #####
    # Iterar sobre todo lo que haya en path
        for file in os.listdir(folder_path):
            
            # Leer la imagen a color y aplicarle el resize
            image_path = os.path.join(folder_path, file)
            image = imread(image_path)
            smallimage = cv2.resize(image, im_size)
            
            # Guardo en X
            X.append(smallimage)
            Y.append(label)

    return np.array(X), np.array(Y)
    


In [112]:
def read_data_test(path, im_size):
    X = []
    Y = []

    for file in os.listdir(path):
            
        # Leer la imagen a color y aplicarle el resize
        image_path = os.path.join(path, file)
        image = imread(image_path)
        smallimage = cv2.resize(image, im_size)
            
        # Guardo en X
        X.append(smallimage)
        #Y.append(label)

    return np.array(X)
    

In [113]:
X_train, y_train = read_data(TRAIN_PATH, IMAGE_SIZE)
X_test = read_data_test(TEST_PATH, IMAGE_SIZE)

print(X_train.shape)
print(X_test.shape)

(28821, 32, 32)
(7066, 32, 32)


In [114]:
X_train, y_train = shuffle(X_train, y_train, random_state=10)

In [115]:
pd.Series(y_train).value_counts(normalize=True)

3    0.248569
4    0.172860
5    0.171333
2    0.142361
0    0.138545
6    0.111204
1    0.015128
dtype: float64

In [116]:
X_train

array([[[ 12,  18,  34, ..., 173, 165, 156],
        [ 15,  28,  42, ..., 159, 159, 161],
        [ 16,  32,  44, ..., 172, 176, 173],
        ...,
        [168, 100,  20, ..., 181, 182, 183],
        [175, 103,  18, ..., 179, 172, 178],
        [138,  41,  23, ..., 182, 179, 183]],

       [[147,  58,  71, ..., 161, 151, 146],
        [ 46,  69,  26, ..., 167, 167, 173],
        [ 41,  80, 101, ..., 128, 157, 166],
        ...,
        [ 52,  54,  50, ...,  64,  70,  31],
        [ 57,  57,  46, ...,  63,  60,  33],
        [ 45,  62,  47, ...,  59,  42,  36]],

       [[254, 254, 254, ..., 253, 255, 255],
        [255, 254, 253, ..., 253, 254, 254],
        [255, 255, 255, ..., 246, 252, 254],
        ...,
        [249, 229, 238, ..., 166, 167, 159],
        [201, 218, 217, ..., 161, 169, 150],
        [229, 229, 220, ..., 163, 166, 147]],

       ...,

       [[203, 199, 189, ...,  17,  15,  14],
        [204, 199, 191, ...,  29,  23,  19],
        [204, 203, 186, ...,  36,  31,  27

In [117]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(32,32,1)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(7, activation='softmax')
])

In [118]:
layers = [
    keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(32,32,1)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(7, activation='sigmoid')
]

#model = keras.Sequential(layers)

In [119]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 30, 30, 64)        640       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 15, 15, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 13, 13, 32)        18464     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 6, 6, 32)         0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 1152)              0         
                                                                 
 dense_38 (Dense)            (None, 128)             

In [120]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [121]:
earlystop = EarlyStopping(patience=5)

In [122]:
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [earlystop]) # batch_size=128,

Epoch 1/20
121/721 [====>.........................] - ETA: 13s - loss: 2.1586 - accuracy: 0.2159

721/721 [==============================] - 18s 24ms/step - loss: 1.8130 - accuracy: 0.2892 - val_loss: 1.6528 - val_accuracy: 0.3710
Epoch 2/20
721/721 [==============================] - 18s 24ms/step - loss: 1.6027 - accuracy: 0.3723 - val_loss: 1.6369 - val_accuracy: 0.3613
Epoch 3/20
721/721 [==============================] - 17s 24ms/step - loss: 1.5389 - accuracy: 0.4027 - val_loss: 1.5527 - val_accuracy: 0.4059
Epoch 4/20
721/721 [==============================] - 18s 25ms/step - loss: 1.4882 - accuracy: 0.4248 - val_loss: 1.5281 - val_accuracy: 0.4128
Epoch 5/20
721/721 [==============================] - 18s 25ms/step - loss: 1.4324 - accuracy: 0.4427 - val_loss: 1.5179 - val_accuracy: 0.4180
Epoch 6/20
721/721 [==============================] - 18s 25ms/step - loss: 1.3681 - accuracy: 0.4728 - val_loss: 1.4828 - val_accuracy: 0.4364
Epoch 7/20
721/721 [==============================] - 18s 25ms/step - loss: 1.3069 - accuracy: 0.5009 - val_loss: 1.4609 - val_accuracy: 0.4538
Epo

In [123]:
df_hist = pd.DataFrame(history.history)

In [124]:
df_hist

,loss,accuracy,val_loss,val_accuracy
0,1.812981,0.289165,1.652795,0.371032
1,1.602653,0.372311,1.636918,0.361318
2,1.538942,0.402715,1.552698,0.405898
3,1.488242,0.424792,1.528107,0.412836
4,1.432376,0.442748,1.517900,0.418040
5,1.368132,0.472762,1.482825,0.436427
6,1.306941,0.500867,1.460908,0.453773
7,1.233397,0.533527,1.452314,0.457762
8,1.161266,0.559768,1.488988,0.452732
9,1.080048,0.589998,1.527271,0.454640


In [125]:
y_pred = model.predict(X_test)

216/221 [============================>.] - ETA: 0s

221/221 [==============================] - 2s 7ms/step


In [126]:
y_pred

array([[4.31411415e-01, 1.28340325e-05, 8.29655386e-04, ...,
        5.33628583e-01, 1.62246972e-02, 1.44809936e-04],
       [8.68339017e-02, 4.18994091e-02, 1.64036334e-01, ...,
        2.11848676e-01, 3.01127285e-01, 2.83452719e-02],
       [1.18765041e-01, 3.28200608e-02, 1.87753692e-01, ...,
        3.35291147e-01, 2.00940460e-01, 8.55562314e-02],
       ...,
       [7.18608618e-01, 2.40949239e-03, 2.55172863e-03, ...,
        1.87139586e-01, 8.00445229e-02, 1.32862115e-04],
       [3.20926100e-01, 7.63823744e-04, 1.53163671e-01, ...,
        4.21650521e-02, 3.80533457e-01, 1.88508462e-02],
       [2.68581491e-02, 1.07596300e-07, 1.03378040e-03, ...,
        3.41395736e-01, 1.03688277e-02, 8.69528833e-07]], dtype=float32)

In [127]:
pred_labels = np.argmax(y_pred, axis=1)
pred_labels

array([4, 5, 4, ..., 0, 5, 3], dtype=int64)

In [128]:
label_pred = [class_names[pred] for pred in pred_labels]
label_pred

['neutral',
 'sad',
 'neutral',
 'fear',
 'angry',
 'sad',
 'neutral',
 'angry',
 'sad',
 'happy',
 'happy',
 'angry',
 'surprise',
 'fear',
 'neutral',
 'sad',
 'happy',
 'angry',
 'fear',
 'angry',
 'angry',
 'neutral',
 'fear',
 'fear',
 'happy',
 'fear',
 'fear',
 'neutral',
 'sad',
 'angry',
 'neutral',
 'sad',
 'happy',
 'surprise',
 'angry',
 'happy',
 'neutral',
 'sad',
 'happy',
 'angry',
 'happy',
 'surprise',
 'sad',
 'surprise',
 'angry',
 'happy',
 'fear',
 'fear',
 'disgust',
 'surprise',
 'angry',
 'happy',
 'fear',
 'neutral',
 'surprise',
 'angry',
 'angry',
 'neutral',
 'angry',
 'happy',
 'happy',
 'neutral',
 'fear',
 'fear',
 'neutral',
 'happy',
 'sad',
 'happy',
 'angry',
 'happy',
 'surprise',
 'sad',
 'happy',
 'neutral',
 'happy',
 'angry',
 'fear',
 'fear',
 'happy',
 'surprise',
 'sad',
 'happy',
 'sad',
 'sad',
 'fear',
 'neutral',
 'happy',
 'angry',
 'neutral',
 'angry',
 'happy',
 'neutral',
 'angry',
 'angry',
 'happy',
 'happy',
 'sad',
 'fear',
 'happ

In [129]:
df_submission = pd.DataFrame({"id_img": test_set["id_img"], "label": label_pred})
df_submission

,id_img,label
0,10052,neutral
1,10065,sad
2,10079,neutral
3,10095,fear
4,10121,angry
...,...,...
7061,9806,happy
7062,9830,happy
7063,9853,angry
7064,9878,sad


In [130]:
df_submission.to_csv("submission_3.csv", index = False)